In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

from itertools import product

import torch
import wandb
import os

In [2]:
cfg_dict = {
    "detector": ['faster_rcnn'],
    "config_name": ["mask_rcnn_r50_fpn_1x_coco.py",
                    "cascade_mask_rcnn_x101_64x4d_fpn_20e_coco.py",
                    "faster_rcnn_r50_fpn_1x_coco.py",
                    "cascade_rcnn_x101_64x4d_fpn_20e_coco.py",
                    "faster_rcnn_r50_fpn_soft_nms_1x_coco.py",
                    "cascade_mask_rcnn_hrnetv2p_w40_20e_coco.py"],
    # 공통
    "cls_loss": ['CrossEntropyLoss'],
    "bbox_loss": ['L1Loss','SmoothL1Loss'],
    # mask rcnn
    "mask_loss": [None],
    
    # Yolo
    "conf_loss": [None],
    "xy_loss": [None],
    "wh_loss": [None]
}

cfg_list = [ v for k,v in cfg_dict.items()]

In [3]:
def _init_cfg_dataset(cfg):
    classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
                   "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
    PREFIX = '../../input/data/'
    # dataset 바꾸기
    cfg.data.train.classes = classes
    cfg.data.train.img_prefix = PREFIX
    cfg.data.train.ann_file = PREFIX + 'train.json'
    cfg.data.train.pipeline[2]['img_scale'] = (512, 512)
    cfg.data.val.classes = classes
    cfg.data.val.img_prefix = PREFIX
    cfg.data.val.ann_file = PREFIX + 'val.json'
    cfg.data.val.pipeline[1]['img_scale'] = (512, 512)
    cfg.data.test.classes = classes
    cfg.data.test.img_prefix = PREFIX
    cfg.data.test.ann_file = PREFIX + 'test.json'
    cfg.data.test.pipeline[1]['img_scale'] = (512, 512)
    
    return cfg

def _init_cfg_num_classes(cfg,detector):
# config head 바꿔주기.
    if "faster_rcnn" == detector:
        cfg.model.roi_head.bbox_head.num_classes = 11
    elif "cascade_rcnn" == detector:
        for di in cfg.model.roi_head.bbox_head:
            di.num_classes = 11
    
    elif "cascade_mask_rcnn" == detector:
        for i in range(len(cfg.train_pipline)):
            if cfg.train_pipeline[i].type == "LoadAnnotations":
                cfg.train_pipeline[i] = dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
            if cfg.train_pipeline[i].type == "Collect":
                cfg.train_pipeline[i] = dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks']),
                
        for di in cfg.model.roi_head.bbox_head:
            di.num_classes = 11
        cfg.model.roi_head.mask_head.num_classes = 11    
    elif "mask_rcnn" == detector:
        for i in range(len(cfg.train_pipline)):
            if cfg.train_pipeline[i].type == "LoadAnnotations":
                cfg.train_pipeline[i] = dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
            if cfg.train_pipeline[i].type == "Collect":
                cfg.train_pipeline[i] = dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks']),
        cfg.model.roi_head.bbox_head.num_classes = 11
        cfg.model.roi_head.mask_head.num_classes = 11
    
    elif "yolo" == detector:
        cfg.model.bbox_head.num_classees = 11
    return cfg

def _init_cfg_losses(cfg,detector,cls_loss=None, bbox_loss=None, mask_loss=None,conf_loss = None,xy_loss = None,wh_loss = None):
    # 1stage: yolo
    if detector == "yolo":
        cfg.model.bbox_head.loss_cls['type']  = cls_loss
        cfg.model.bbox_head.loss_conf['type']  = conf_loss
        cfg.model.bbox_head.loss_xy['type']  = xy_loss
        cfg.model.bbox_head.loss_wh['type']  = wh_loss
    
    # 2stage: faster, mask, cascade rcnn
    else:
        cfg.model.rpn_head.loss_cls['type']  = cls_loss
        cfg.model.rpn_head.loss_bbox['type']  = bbox_loss
        
        # cascade rcnn는 bbox_head 여러 개
        if isinstance(cfg.model.roi_head.bbox_head,list):
            for bbox_head in cfg.model.roi_head.bbox_head:
                bbox_head.loss_cls['type']  = cls_loss
                bbox_head.loss_bbox['type']  = bbox_loss
        # 나머지는 bbox_head가 한개
        else:
            cfg.model.roi_head.bbox_head.loss_cls['type']  = cls_loss
            cfg.model.roi_head.bbox_head.loss_bbox['type']  = bbox_loss
        if mask_loss is not None:
            cfg.model.roi_head.mask_head.loss_mask['type']  = mask_loss
    return cfg

In [4]:
def grid_train(cfg_list):
    for comb in list(product(*cfg_list)):
        detector, config_name, cls_loss, bbox_loss, mask_loss, conf_loss, xy_loss, wh_loss  = comb
        
        if detector != config_name[:len(detector)]:
            continue
        print('-----------'*10)
        print(f'{detector},{config_name} start!!!')
        print('-----------'*10)
        # config 불러오기.
        cfg = Config.fromfile(os.path.join('./configs',detector, config_name))
        
        cfg = _init_cfg_dataset(cfg)
        cfg = _init_cfg_num_classes(cfg,detector)
        cfg = _init_cfg_losses(cfg,detector,cls_loss, bbox_loss, mask_loss,conf_loss,xy_loss,wh_loss)
        
        cfg.data.samples_per_gpu = 4
        cfg.seed=2020
        cfg.gpu_ids = [0]
        cfg.work_dir = os.path.join('./work_dirs',config_name+'trash')
        cfg.runner['max_epochs'] = 10
        cfg.checkpoint_config['interval'] = 1
        # 여기 이름도 원하는 이름으로 바꾸세요
        wandb_run_name = f'{config_name}___{cls_loss}__{bbox_loss}__{mask_loss}' if mask_loss is not None else  f'{config_name}___{cls_loss}__{bbox_loss}'
        
        cfg.log_config.hooks.append(dict(
            type='WandbLoggerHook',
            init_kwargs=dict(
                project="Seungwoo",
                name = wandb_run_name
                )
        ))
        
        cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
        cfg.fp16 = dict(loss_scale=512.)

        model = build_detector(cfg.model)
        
        datasets = [build_dataset(cfg.data.train)]
        
        train_detector(model, datasets[0], cfg, distributed=False, validate=True)
        
        torch.cuda.empty_cache()
    
    result = os.system("bash del.sh " + config_name + 'trash')    
    
    if result == 0:
        print("성공적으로 지워짐.")
        

In [5]:
grid_train(cfg_list)

--------------------------------------------------------------------------------------------------------------
faster_rcnn,faster_rcnn_r50_fpn_1x_coco.py start!!!
--------------------------------------------------------------------------------------------------------------


2021-05-14 10:25:52,442 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-14 10:25:52,443 - mmdet - INFO - Use load_from_torchvision loader
2021-05-14 10:25:52,681 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



loading annotations into memory...
Done (t=3.34s)
creating index...
index created!
loading annotations into memory...


2021-05-14 10:25:59,688 - mmdet - INFO - Start running, host: root@550dc0fe0f09, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/faster_rcnn_r50_fpn_1x_coco.pytrash
2021-05-14 10:25:59,688 - mmdet - INFO - workflow: [('train', 1)], max: 10 epochs


Done (t=0.80s)
creating index...
index created!


...

2021-05-14 12:59:12,521 - mmdet - INFO - Now best checkpoint is epoch_9.pth.Best bbox_mAP is 0.1570
2021-05-14 12:59:12,527 - mmdet - INFO - Epoch(val) [9][655]	bbox_mAP: 0.1570, bbox_mAP_50: 0.2730, bbox_mAP_75: 0.1540, bbox_mAP_s: 0.0330, bbox_mAP_m: 0.1460, bbox_mAP_l: 0.2910, bbox_mAP_copypaste: 0.157 0.273 0.154 0.033 0.146 0.291


DONE (t=1.23s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.157
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.273
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.154
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.033
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.146
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.291
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.333
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.333
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.333
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.098
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.309
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.578


2021-05-14 12:59:29,448 - mmdet - INFO - Epoch [10][50/655]	lr: 2.000e-03, eta: 0:02:59, time: 0.338, data_time: 0.050, memory: 1497, loss_rpn_cls: 0.0315, loss_rpn_bbox: 0.0067, loss_cls: 0.2672, acc: 91.6064, loss_bbox: 0.1312, loss: 0.4366, grad_norm: 1.7176
2021-05-14 12:59:43,912 - mmdet - INFO - Epoch [10][100/655]	lr: 2.000e-03, eta: 0:02:44, time: 0.289, data_time: 0.006, memory: 1497, loss_rpn_cls: 0.0245, loss_rpn_bbox: 0.0046, loss_cls: 0.2217, acc: 92.6914, loss_bbox: 0.1080, loss: 0.3588, grad_norm: 1.6090
2021-05-14 12:59:58,435 - mmdet - INFO - Epoch [10][150/655]	lr: 2.000e-03, eta: 0:02:30, time: 0.290, data_time: 0.006, memory: 1497, loss_rpn_cls: 0.0232, loss_rpn_bbox: 0.0047, loss_cls: 0.2289, acc: 92.6123, loss_bbox: 0.1092, loss: 0.3660, grad_norm: 1.6370
2021-05-14 13:00:12,929 - mmdet - INFO - Epoch [10][200/655]	lr: 2.000e-03, eta: 0:02:15, time: 0.290, data_time: 0.006, memory: 1497, loss_rpn_cls: 0.0242, loss_rpn_bbox: 0.0047, loss_cls: 0.2166, acc: 92.9033, 

[>>>>>>>>>>>>                 ] 284/655, 22.5 task/s, elapsed: 13s, ETA:    17s

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:

a = os.system("echo 1")
a

0